## Избавление от полисемии в предобученном W2V

Возьмем для примера 2 предложения, в каждом из которых будет содержаться определенное слово, но в разных значениях. Натренируем модель и покажем, что возникает проблема полисемии.

In [1]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
spacy.load("en")

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import gensim
from gensim.models.phrases import Phrases, Phraser
import multiprocessing
from gensim.models import Word2Vec

INFO - 13:29:28: 'pattern' package not found; tag filters are not available for English


Предложения возьмем следующие:
    
1) “My friend is considering to take a loan from a bank.”

2) “Rainfall caused Rhine river to overflow it’s bank. ”

In [2]:
tokens = [["my", "friend", "considering", "to", "take", "a", "loan", "from", "a", "bank"],
 ["rainfall", "caused", "neckar", "river", "to", "overflow", "bank", "", "", ""]]

In [3]:
phrases = Phrases(tokens, min_count=30, progress_per=10000) # train full Phrases model
bigram = Phraser(phrases) # cut down memory consumption of Phrases
sentences = bigram[tokens]

INFO - 13:29:29: collecting all words and their counts
INFO - 13:29:29: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 13:29:29: collected 32 word types from a corpus of 20 words (unigram + bigrams) and 2 sentences
INFO - 13:29:29: using 32 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 13:29:29: source_vocab length 32
INFO - 13:29:29: Phraser built with 0 phrasegrams


In [4]:
w2v_model = Word2Vec(min_count=1,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=0, # 20
                     workers=-1)

In [5]:
w2v_model.build_vocab(tokens, progress_per=100)
w2v_model.train(tokens, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
# w2v_model=w2v_model.wv.init_sims(replace=True)

INFO - 13:29:29: collecting all words and their counts
INFO - 13:29:29: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:29:29: collected 15 word types from a corpus of 20 raw words and 2 sentences
INFO - 13:29:29: Loading a fresh vocabulary
INFO - 13:29:29: effective_min_count=1 retains 15 unique words (100% of original 15, drops 0)
INFO - 13:29:29: effective_min_count=1 leaves 20 word corpus (100% of original 20, drops 0)
INFO - 13:29:29: deleting the raw counts dictionary of 15 items
INFO - 13:29:29: sample=6e-05 downsamples 15 most-common words
INFO - 13:29:29: downsampling leaves estimated 0 word corpus (3.0% of prior 20)
INFO - 13:29:29: estimated required memory for 15 words and 300 dimensions: 25500 bytes
INFO - 13:29:29: resetting layer weights
INFO - 13:29:29: training model with -1 workers on 15 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=0 window=2
INFO - 13:29:29: EPOCH - 1 : training on 0 raw words (0 effective words) took

INFO - 13:29:29: EPOCH - 38 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 38 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 39 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 39 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 40 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 40 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 41 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 41 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 42 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 42 : supplied example count (0) did not equal expected count (2)
INFO - 13:

WARNING - 13:29:29: EPOCH - 79 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 80 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 80 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 81 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 81 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 82 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 82 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 83 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 13:29:29: EPOCH - 83 : supplied example count (0) did not equal expected count (2)
INFO - 13:29:29: EPOCH - 84 : training on 0 raw words (0 effective words) took 0.0s, 0 effective words/s
WARNING - 

(0, 0)

Модель мы натренировали, теперь покажем проблему полисемии для слова "bank". Это слово может быть использовано как в значении "банк", так и в значении "берег озера". Тем не менее в Word2vec этому слову для всех его значений будет поставлен в соответствие только один вектор размером, указанным нами ранее (size=300):

In [6]:
w2v_model.wv['bank'][:10]

array([-1.3996725e-03, -1.1365307e-03, -2.7173077e-04,  4.7763996e-04,
        1.6365285e-03,  3.5416731e-05,  6.3574029e-04, -1.3060764e-04,
       -9.2497311e-04, -1.3173001e-03], dtype=float32)

In [26]:
tokens[0][9], tokens[1][6]

('bank', 'bank')

Вектор для слова "bank" = "банк" из первого предложения:

In [7]:
w2v_model.wv.get_vector(tokens[0][9])[:10]

array([-1.3996725e-03, -1.1365307e-03, -2.7173077e-04,  4.7763996e-04,
        1.6365285e-03,  3.5416731e-05,  6.3574029e-04, -1.3060764e-04,
       -9.2497311e-04, -1.3173001e-03], dtype=float32)

Вектор для слова "bank" = "берег озера" из второго предложения:

In [8]:
w2v_model.wv.get_vector(tokens[1][6])[:10]

array([-1.3996725e-03, -1.1365307e-03, -2.7173077e-04,  4.7763996e-04,
        1.6365285e-03,  3.5416731e-05,  6.3574029e-04, -1.3060764e-04,
       -9.2497311e-04, -1.3173001e-03], dtype=float32)

Как видим, векторы совпадают (для краткости было выведено 10 первых координат, но можно было сравнить также и остальные 290). Таким образом, нескольким значениям одного слова соответствует один вектор.

Что можем сделать:

* Используем несколько представлений слова, чтобы обнаружить полисемию
* Кластеризуем контексты, чтобы обнаружить полисемию; затем переприсвоим лейблы словам, чтобы у каждого смысла для слова был свой вектор
* Находим альтернативные представления для слов при помощи постобработки нормальных Word2vec векторов

Источник: https://stackoverflow.com/questions/51330549/using-word2vec-for-polysemy-solving-problems

В нашем примере мы используем модель ELMo (Embeddings from Language Models):

* Если Word2vec использует на входе слова, то ELMo берет для этого комбинации букв
* В Word2vec для построения ембедингов слов используется 1 скрытый слой (с выходным - 2 слоя), в то время как в ELMo для сети слоя 3: (1) Сверточные нейронные сети (CNN), (2) Двунаправленная сеть с долгой краткосрочной памятью (LSTM), (3) Ембединги
* ELMo строит ембединги динамически, ориентируясь на близлежащие буквы

На вход первому слою (CNN) подается последовательность комбинаций букв из предложения, Bi-directed LSTM (слой 2) для каждого слова учитывает его окружение слева и справа, чтобы затем ембединги могли объединить результат уже с учетом контекста слова.


In [18]:
import tensorflow as tf
import tensorflow_hub as hub

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

In [19]:
tokens_length = [10, 7]
embeddings = elmo(
        inputs={
            "tokens": tokens,
            "sequence_len": tokens_length
        },
        signature="tokens",
        as_dict=True)["elmo"]

I0717 14:59:47.388398 4643038656 saver.py:1499] Saver not created because there are no variables in the graph to restore


In [23]:
session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])
elmo_embeddings = session.run(embeddings)

In [24]:
elmo_embeddings[0][9][:10]

array([-0.02096611,  0.23486409,  0.00171002,  0.21556687,  0.68794274,
        0.6366527 , -0.18037347,  0.2796358 , -0.04736871,  0.41843903],
      dtype=float32)

In [25]:
elmo_embeddings[1][6][:10]

array([ 0.3807178 ,  0.60629106,  0.22663993,  0.07565593,  0.2112593 ,
        0.32979885, -0.3203162 , -0.11011641,  0.48975784, -0.00121106],
      dtype=float32)

Как видно уже из первых координат векторов, контекст учесть удалось: для первого и второго предложения слово "bank" имеет свой вектор для своего контекста.